# Get zonal stats 

In [2]:
import ibis
import ibis.selectors as s
from ibis import _
import fiona
import geopandas as gpd
import rioxarray
from shapely.geometry import box

import rasterio
from rasterio.mask import mask
from rasterstats import zonal_stats
import pandas as pd
from joblib import Parallel, delayed

con = ibis.duckdb.connect()
con.load_extension("spatial")
threads = -1

In [19]:
# cropping US data to only CA 
def crop_raster_to_bounds(tif_file, vector_gdf):
    with rasterio.open(tif_file) as src:
        # Get California's bounding box in the same CRS as the raster
        california_bounds = vector_gdf.total_bounds
        california_bounds = rasterio.coords.BoundingBox(
            *california_bounds
        )
        # Crop the raster to the California bounding box
        out_image, out_transform = mask(src, [california_bounds], crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })
        print("Unique values in cropped raster:", np.unique(out_image))

    return out_image, out_meta


In [20]:
def big_zonal_stats(vec_file, tif_file, stats, col_name, n_jobs, verbose=10, timeout=10000):
    gdf = gpd.read_parquet(vec_file)
    if gdf.crs is None:
        gdf = gdf.set_crs("EPSG:4326")
    gdf = gdf.rename(columns={"geom": "geometry"})
    gdf = gdf.set_geometry("geometry")
    gdf = gdf[gdf["geometry"].notna()].copy()

    with rasterio.open(tif_file) as src:
        raster_crs = src.crs
        gdf = gdf.to_crs(raster_crs)  # Transform vector to raster CRS
        
        # CA bounding box + convert it to a polygon in raster CRS
        california_polygon = box(*gdf.total_bounds)
        
        out_image, out_transform = mask(src, [california_polygon], crop=True, nodata=src.nodata)

        # If raster is 3D, select the first band
        if out_image.ndim == 3:
            out_image = out_image[0]

    # compute zonal statistics for each geometry slice
    def get_stats(geom_slice):
        geom = [geom_slice.geometry]
        stats_result = zonal_stats(
            geom, out_image, stats=stats, affine=out_transform, all_touched=True, nodata=src.nodata
        )
        return stats_result[0] if stats_result and stats_result[0].get("mean") is not None else {'mean': None}

    output = [get_stats(row) for row in gdf.itertuples()]
    gdf[col_name] = [res['mean'] for res in output]

    return gdf

In [8]:
# aws s3 cp s3://vizzuality/hfp-100/hfp_2021_100m_v1-2_cog.tif . --endpoint-url=https://data.source.coop
# aws s3 cp s3://vizzuality/lg-land-carbon-data/natcrop_bii_100m_cog.tif . --endpoint-url=https://data.source.coop
# aws s3 cp s3://vizzuality/lg-land-carbon-data/natcrop_fii_100m_cog.tif . --endpoint-url=https://data.source.coop
# aws s3 cp s3://vizzuality/lg-land-carbon-data/natcrop_expansion_100m_cog.tif . --endpoint-url=https://data.source.coop
# aws s3 cp s3://vizzuality/lg-land-carbon-data/natcrop_reduction_100m_cog.tif . --endpoint-url=https://data.source.coop
# aws s3 cp s3://cboettig/carbon/cogs/irrecoverable_c_total_2018.tif . --endpoint-url=https://data.source.coop
# aws s3 cp s3://cboettig/carbon/cogs/manageable_c_total_2018.tif . --endpoint-url=https://data.source.coop
# ! aws s3 cp s3://cboettig/justice40/disadvantaged-communities.parquet . --endpoint-url=https://data.source.coop
# minio/shared-biodiversity/redlist/cog/combined_sr_2022.tif
# /home/rstudio/minio/shared-biodiversity/redlist/cog/combined_rwr_2022.tif
# ! aws s3 cp s3://cboettig/social-vulnerability/svi2020_us_tract.parquet . --endpoint-url=https://data.source.coop


In [21]:
%%time
tif_file = 'SpeciesRichness_All.tif'
vec_file = "/home/rstudio/github/ca-30x30/ca2024-30m.parquet"
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'], col_name = "richness", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")


CPU times: user 1min 52s, sys: 5.01 s, total: 1min 57s
Wall time: 1min 57s


In [22]:
%%time
tif_file = 'RSR_All.tif'
vec_file = './cpad-stats-temp.parquet'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],
                      col_name = "rsr", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")

CPU times: user 1min 50s, sys: 4.47 s, total: 1min 54s
Wall time: 1min 54s


In [23]:
%%time
tif_file = 'deforest_carbon_100m_cog.tif'
vec_file = './cpad-stats-temp.parquet'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'], 
                     col_name = "deforest_carbon", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")


/opt/venv/lib/python3.10/site-packages/rasterstats/io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


CPU times: user 1min 58s, sys: 4.93 s, total: 2min 3s
Wall time: 2min 3s


In [24]:
%%time
tif_file = 'natcrop_bii_100m_cog.tif'
vec_file = './cpad-stats-temp.parquet'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'], 
                     col_name = "biodiversity_intactness_loss", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")


CPU times: user 1min 53s, sys: 4.81 s, total: 1min 58s
Wall time: 1min 58s


In [25]:
%%time
tif_file = 'natcrop_fii_100m_cog.tif'
vec_file = './cpad-stats-temp.parquet'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],
                     col_name = "forest_integrity_loss", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")



CPU times: user 1min 53s, sys: 4.9 s, total: 1min 58s
Wall time: 1min 58s


In [7]:
%%time
tif_file = 'natcrop_expansion_100m_cog.tif'
vec_file = './cpad-stats-temp.parquet'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],  col_name = "crop_expansion", n_jobs=threads, verbose=0)
gpd.GeoDataFrame(df, geometry="geometry").to_parquet("cpad-stats-temp.parquet")


/opt/conda/lib/python3.12/site-packages/rasterstats/io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


CPU times: user 3min 13s, sys: 55 s, total: 4min 8s
Wall time: 4min 8s


In [ ]:
%%time
tif_file = 'natcrop_reduction_100m_cog.tif'
vec_file = './cpad-stats-temp.parquet'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],  col_name = "crop_reduction", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")


In [ ]:
%%time
tif_file = 'irrecoverable_c_total_2018.tif'
vec_file = './cpad-stats-temp.parquet'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],  col_name = "irrecoverable_carbon", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")



In [ ]:
%%time
tif_file = 'manageable_c_total_2018.tif'
vec_file = './cpad-stats-temp.parquet'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],  col_name = "manageable_carbon", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")


In [ ]:
%%time
tif_file = 'combined_rwr_2022.tif'
vec_file = './cpad-stats-temp.parquet'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],  col_name = "all_species_rwr", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")


In [ ]:
%%time
tif_file = 'combined_sr_2022.tif'
vec_file = './cpad-stats-temp.parquet'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],  col_name = "all_species_richness", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")


In [ ]:
%%time
tif_file = 'combined_sr_2022.tif'
vec_file = './cpad-stats-temp.parquet'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],  col_name = "all_species_richness", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")


In [20]:
%%time
tif_file = 'hfp_2021_100m_v1-2_cog.tif'
vec_file = './cpad-stats-temp.parquet'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],  col_name = "human_impact", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")


CPU times: user 3min 16s, sys: 57 s, total: 4min 13s
Wall time: 4min 12s


# Convert vector to tif 

In [24]:
import geopandas as gpd
import numpy as np
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds

def get_geotiff(gdf, output_file,col):
    gdf = gdf.set_geometry("geometry")
    gdf = gdf.set_crs("EPSG:4326")
    print(gdf.crs)

    # Set raster properties
    minx, miny, maxx, maxy = gdf.total_bounds  # Get the bounds of the geometry
    pixel_size = 0.01  # Define the pixel size in units of the CRS
    width = int((maxx - minx) / pixel_size)
    height = int((maxy - miny) / pixel_size)
    transform = from_bounds(minx, miny, maxx, maxy, width, height)
    
    # Define rasterization with continuous values
    shapes = ((geom, value) for geom, value in zip(gdf.geometry, gdf[col]))
    raster = rasterize(
        shapes,
        out_shape=(height, width),
        transform=transform,
        fill=0.0,  # Background value for areas outside the geometry
        dtype="float32"  # Set data type to handle continuous values
    )
    print("Unique values in raster:", np.unique(raster))

    # Define GeoTIFF metadata
    out_meta = {
        "driver": "GTiff",
        "height": height,
        "width": width,
        "count": 1,
        "dtype": raster.dtype,
        "crs": gdf.crs,
        "transform": transform,
        "compress": "deflate"  # Use compression to reduce file size
    }
    
    # Write to a GeoTIFF file with COG options
    with rasterio.open(output_file, "w", **out_meta) as dest:
        dest.write(raster, 1)
        dest.build_overviews([2, 4, 8, 16], rasterio.enums.Resampling.average)
        dest.update_tags(1, TIFFTAG_RESOLUTION_UNIT="Meter")


In [25]:
# clean up SVI data
svi_df =  (con
 .read_parquet("svi2020_us_tract.parquet")
 .select("RPL_THEMES","RPL_THEME1","RPL_THEME2","RPL_THEME3","RPL_THEME4","Shape")
 .rename(SVI = "RPL_THEMES", socioeconomic = "RPL_THEME1", 
         household_char = "RPL_THEME2", racial_ethnic_minority =  "RPL_THEME3",
         housing_transit = "RPL_THEME4", geometry = "Shape")
.cast({"geometry":"geometry"})
)
svi_df.execute().to_parquet("svi2020_us_tract_clean.parquet")


In [27]:
gdf = gpd.read_parquet("svi2020_us_tract_clean.parquet")
svi = gdf[['SVI','geometry']]
socio = gdf[['socioeconomic','geometry']]
house = gdf[['household_char','geometry']]
minority = gdf[['racial_ethnic_minority','geometry']]
transit = gdf[['housing_transit','geometry']]

#convert SVI parquet to tif
get_geotiff(svi,"svi.tif","SVI")
get_geotiff(socio,"svi_socioeconomic.tif","socioeconomic")
get_geotiff(house,"svi_household.tif","household_char")
get_geotiff(minority,"svi_minority.tif","racial_ethnic_minority")
get_geotiff(transit,"svi_transit.tif","housing_transit")

EPSG:4326
Unique values in raster: [-9.990e+02  0.000e+00  1.000e-04 ...  9.998e-01  9.999e-01  1.000e+00]
EPSG:4326
Unique values in raster: [-9.990e+02  0.000e+00  4.000e-04 ...  9.998e-01  9.999e-01  1.000e+00]
EPSG:4326
Unique values in raster: [-9.990e+02  0.000e+00  3.000e-04 ...  9.998e-01  9.999e-01  1.000e+00]
EPSG:4326
Unique values in raster: [-9.990e+02  0.000e+00  2.400e-03 ...  9.943e-01  9.952e-01  9.959e-01]
EPSG:4326
Unique values in raster: [-9.990e+02  0.000e+00  9.000e-03 ...  9.998e-01  9.999e-01  1.000e+00]


In [28]:
%%time
tif_file = 'svi.tif'
vec_file = './cpad-stats-temp.parquet'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],  col_name = "SVI", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")



CPU times: user 3min 26s, sys: 56.6 s, total: 4min 23s
Wall time: 4min 27s


In [29]:
%%time
vec_file = './cpad-stats-temp.parquet'
tif_file = 'svi_socioeconomic.tif'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],  col_name = "socioeconomic_status", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")



CPU times: user 3min 22s, sys: 56.4 s, total: 4min 18s
Wall time: 4min 27s


In [30]:
%%time
vec_file = './cpad-stats-temp.parquet'
tif_file = 'svi_household.tif'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],  col_name = "household_char", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")



CPU times: user 3min 11s, sys: 54.3 s, total: 4min 5s
Wall time: 4min 5s


In [31]:
%%time
vec_file = './cpad-stats-temp.parquet'
tif_file = 'svi_minority.tif'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],  col_name = "racial_ethnic_minority", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")


CPU times: user 3min 11s, sys: 54.1 s, total: 4min 5s
Wall time: 4min 5s


In [32]:
%%time
vec_file = './cpad-stats-temp.parquet'
tif_file = 'svi_transit.tif'
df = big_zonal_stats(vec_file, tif_file, stats = ['mean'],  col_name = "housing_transit", n_jobs=threads, verbose=0).to_parquet("cpad-stats-temp.parquet")


CPU times: user 3min 13s, sys: 54.7 s, total: 4min 8s
Wall time: 4min 7s


In [3]:
## clean up

con = ibis.duckdb.connect(extensions=["spatial"])
ca_geom = con.read_parquet("ca2024-30m.parquet").cast({"geom":"geometry"}).select("id","geom")



ca = (con
      .read_parquet("cpad-stats-temp.parquet")
      .mutate(richness = _.richness.round(3))
      .mutate(rsr = _.rsr.round(3))
      .mutate(deforest_carbon = _.deforest_carbon.round(3))
      .mutate(biodiversity_intactness_loss = _.biodiversity_intactness_loss.round(3))
      .mutate(forest_integrity_loss = _.forest_integrity_loss.round(3))
      .cast({"crop_expansion": "int64"})
      .cast({"crop_reduction": "int64"})
      .cast({"manageable_carbon": "int64"})
      .cast({"irrecoverable_carbon": "int64"})
      .mutate(all_species_rwr = _.all_species_rwr.round(3))
      .mutate(all_species_richness = _.all_species_richness.round(3))
      .mutate(human_impact = _.human_impact.round(3))
      .mutate(svi = _.SVI.round(3))
      .mutate(svi_socioeconomic_status = _.socioeconomic_status.round(3))
      .mutate(svi_household_char = _.household_char.round(3))
      .mutate(svi_racial_ethnic_minority = _.racial_ethnic_minority.round(3))
      .mutate(svi_housing_transit = _.housing_transit.round(3))
      .drop("geometry","__index_level_0__")
      # .rename(geom = "geometry")
      # .cast({"geom":"geometry"})
      # .mutate(geom=_.geom.convert('EPSG:3857', 'EPSG:4326'))
      .join(ca_geom, "id", how = "inner")
      .drop("SVI", "socioeconomic_status","household_char","racial_ethnic_minority","housing_transit" )

     )
# 
ca.head(5).execute()


,established,reGAP,name,access_type,manager,manager_type,Easement,Acres,id,type,...,all_species_rwr,all_species_richness,crop_expansion,human_impact,svi,svi_socioeconomic_status,svi_household_char,svi_racial_ethnic_minority,svi_housing_transit,geom
0,2024,2,Six Rivers National Forest,Open Access,United States Forest Service,Federal,0,0.191763,100001,Land,...,0.355,346.0,0,2339.00,0.521,0.29,0.522,0.428,0.816,"MULTIPOLYGON Z (((-123.94358 41.95869 0, -123...."
1,2024,1,Six Rivers National Forest,Open Access,United States Forest Service,Federal,0,0.247565,100002,Land,...,0.355,346.0,0,870.50,0.521,0.29,0.522,0.428,0.816,"MULTIPOLYGON Z (((-123.98793 41.94847 0, -123...."
2,2024,1,Six Rivers National Forest,Open Access,United States Forest Service,Federal,0,1.046992,100003,Land,...,0.355,346.0,0,429.00,0.521,0.29,0.522,0.429,0.816,"MULTIPOLYGON Z (((-123.87957 41.97172 0, -123...."
3,2024,1,Six Rivers National Forest,Open Access,United States Forest Service,Federal,0,0.293964,100004,Land,...,0.355,346.0,0,3907.00,0.521,0.29,0.522,0.428,0.816,"MULTIPOLYGON Z (((-123.84466 41.99139 0, -123...."
4,2024,1,Six Rivers National Forest,Open Access,United States Forest Service,Federal,0,0.912564,100005,Land,...,0.355,346.0,0,698.25,0.521,0.29,0.522,0.428,0.816,"MULTIPOLYGON Z (((-123.86194 41.98176 0, -123...."


# Save as PMTiles + Upload data

In [4]:
import subprocess
import os
from huggingface_hub import HfApi, login
import streamlit as st

login(st.secrets["HF_TOKEN"])
# api = HfApi(add_to_git_credential=False)
api = HfApi()

def hf_upload(file, repo_id):
    info = api.upload_file(
            path_or_fileobj=file,
            path_in_repo=file,
            repo_id=repo_id,
            repo_type="dataset",
        )
def generate_pmtiles(input_file, output_file, max_zoom=12):
    # Ensure Tippecanoe is installed
    if subprocess.call(["which", "tippecanoe"], stdout=subprocess.DEVNULL) != 0:
        raise RuntimeError("Tippecanoe is not installed or not in PATH")

    # Construct the Tippecanoe command
    command = [
        "tippecanoe",
        "-o", output_file,
        "-zg",
        "--extend-zooms-if-still-dropping",
        "--force",
        "--projection", "EPSG:4326",  
        "-L","layer:"+input_file,
    ]
    # Run Tippecanoe
    try:
        subprocess.run(command, check=True)
        print(f"Successfully generated PMTiles file: {output_file}")
    except subprocess.CalledProcessError as e:
        print(f"Error running Tippecanoe: {e}")



Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [5]:
gdf = ca.execute().set_crs("EPSG:4326")
gdf.to_file("cpad-stats.geojson")

generate_pmtiles("cpad-stats.geojson", "cpad-stats.pmtiles")
hf_upload("cpad-stats.pmtiles", "boettiger-lab/ca-30x30")

gdf.to_parquet("cpad-stats.parquet")
hf_upload("cpad-stats.parquet", "boettiger-lab/ca-30x30")



cpad-stats.geojson:6: ignoring dimensions beyond two: in JSON object [-123.94358428532209,41.95869046159588,0]
cpad-stats.geojson:6: ignoring dimensions beyond two: in JSON object {"type":"Feature","properties":{"established":2024,"reGAP":2,"name":"Six Rivers National Forest","access_type":"Open Access","manager":"United States Forest Service","manager_type":"Federal","Easement":0,"Acres":0.19176257,"id":100001,"type":"Land","richness":4,"rsr":0.007,"deforest_carbon":0,"biodiversity_intactness_loss":0,"forest_integrity_loss":0,"crop_reduction":0,"irrecoverable_carbon":4,"manageable_carbon":85,"all_species_rwr":0.355,"all_species_richness":346,"crop_expansion":0,"human_...
81196 features, 78827308 bytes of geometry and attributes, 2702235 bytes of string pool, 0 bytes of vertices, 0 bytes of nodes
Choosing a maxzoom of -z10 for features typically 1205 feet (368 meters) apart, and at least 78 feet (24 meters) apart
Choosing a maxzoom of -z13 for resolution of about 39 feet (11 meters) wi

Successfully generated PMTiles file: cpad-stats.pmtiles


cpad-stats.pmtiles:   0%|          | 0.00/95.3M [00:00<?, ?B/s]

cpad-stats.parquet:   0%|          | 0.00/204M [00:00<?, ?B/s]